# "Урок 4. Парсинг HTML. XPath"

## 1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news

### Для парсинга использовать xpath. Структура данных должна содержать:
#### - название источника,
#### - наименование новости,
#### - ссылку на новость,
#### - дата публикации

## 2)Сложить все новости в БД

In [10]:
import requests
import re
from lxml import html
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime

In [5]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36'}

In [13]:
class ManagerMongoDB:
    '''менеджер mongoDB'''

    client = MongoClient('localhost', 27017)  # Объявляем mongoDB
    db = client['news']  # создание БД

    def __init__(self, data):
        '''приём данных'''

        self.data = data

    def into_mongoDB(self):
        '''добавление данных в MongoDB'''

        news = self.db.news

        for d in self.data:
            news.update_one({'_id': d['_id']}, {'$set': d}, upsert=True)

        return news

    def show_result(self):
        '''показать результат'''

        result = []
        for i in self.into_mongoDB().find({}):
            result.append(i)

        return result

In [23]:
class NMParser:
    '''парсер news.mail.ru'''

    main_link = 'https://news.mail.ru/'

    def __init__(self, agent):
        '''получение user agent'''

        self.header = agent

    def get_html(self):
        '''получаем доступ к странице'''

        response = requests.get(self.main_link, headers=self.header)
        root = html.fromstring(response.text)

        return root

    def get_news(self, root):
        '''сбор названия и ссылки'''

        main = "//a[contains(@class,'js-topnews__item')]"

        path_lists = root.xpath(f"{main}/@href"), \
                     root.xpath("//ul[contains(@class,'list_half')]/li/a/@href")
        paths = []
        for path_list in path_lists:
            for path in path_list:
                paths.append(path)

        name_lists = root.xpath(f"{main}/span/span[contains(@class,'photo__title')]/text()"), \
                     root.xpath("//ul[contains(@class,'list_half')]/li/a/text()")
        names = []
        for name_list in name_lists:
            for name in name_list:
                names.append(name)

        return names, paths

    def edit_name(self, name):
        '''редактор имени'''

        pat = '\xa0'
        pattern = re.compile(pat)
        if pat in name:
            return re.sub(pattern, ' ', name)
        else:
            return name

    def time_source(self, link):
        '''собираем дату и источник'''

        response = requests.get(link, headers=self.header)
        root = html.fromstring(response.text)

        # источник
        try:
            source = root.xpath("//a[@class='link color_gray breadcrumbs__link']/span[@class='link__text']/text()")[0]
        except IndexError as e:
            source = None

        # дата публикации
        pattern = re.compile('\+[\d:]+')
        try:
            date_time = root.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
            date_time = re.sub(pattern, '', re.sub('T', ' ', date_time))
        except IndexError as e:
            date_time = None

        return source, date_time

    def get_id(self, link):
        '''собираем id'''

        pattern = re.compile('/(\d+)/')
        
        return re.findall(pattern, link)

    def result(self):
        '''собираем данные'''

        root = self.get_html()
        news = self.get_news(root)
        
        result = []

        for n in range(len(news[0])):
            data_news = {}

            name = self.edit_name(news[0][n])
            link = self.main_link + news[1][n]
            source = self.time_source(link)[0]
            date_time = self.time_source(link)[1]
            id = self.get_id(link)[0]

            data_news['name'] = name
            data_news['link'] = link
            data_news['source'] = source
            data_news['date_time'] = date_time
            data_news['_id'] = id

            result.append(data_news)

        return result

In [27]:
NMdata = NMParser(header).result()

pprint(NMdata)
print(f'\nКоличество объектов: {len(NMdata)}')

[{'_id': '41450650',
  'date_time': '2020-04-19 12:14:39',
  'link': 'https://news.mail.ru//society/41450650/',
  'name': 'В России за сутки заразились коронавирусом 6060 человек',
  'source': 'Ведомости'},
 {'_id': '41453467',
  'date_time': '2020-04-19 17:03:58',
  'link': 'https://news.mail.ru//society/41453467/',
  'name': 'Ученые назвали тревожный симптом коронавируса',
  'source': 'РИА Новости'},
 {'_id': '41453715',
  'date_time': '2020-04-19 17:21:55',
  'link': 'https://news.mail.ru//incident/41453715/',
  'name': 'В Москве арестован грузинский криминальный авторитет',
  'source': 'Коммерсантъ'},
 {'_id': '41450332',
  'date_time': None,
  'link': 'https://news.mail.ru/https://sportmail.ru/news/figure-skating/41450332/',
  'name': 'Уехавший в США русский фигурист рассказал о панике американцев',
  'source': None},
 {'_id': '41453332',
  'date_time': '2020-04-19 16:43:05',
  'link': 'https://news.mail.ru//politics/41453332/',
  'name': 'Российские военные испытали танк «Армата»

In [28]:
mongo = ManagerMongoDB(NMdata).show_result()

pprint(mongo)
print(f'\nКоличество объектов: {len(mongo)}')

[{'_id': '41447768',
  'date_time': '2020-04-18 23:46:54',
  'link': 'https://news.mail.ru//society/41447768/',
  'name': 'Пасхальное богослужение в храме Христа Спасителя',
  'source': 'ТАСС'},
 {'_id': '41445652',
  'date_time': '2020-04-18 20:50:28',
  'link': 'https://news.mail.ru//society/41445652/',
  'name': 'Коронавирус COVID-19: главные события 18 апреля',
  'source': 'Новости Mail.ru'},
 {'_id': '41446112',
  'date_time': '2020-04-18 19:58:11',
  'link': 'https://news.mail.ru//economics/41446112/',
  'name': 'Туристический сезон в Турции может быть открыт в конце мая',
  'source': 'Коммерсантъ'},
 {'_id': '41444458',
  'date_time': '2020-04-18 17:36:29',
  'link': 'https://news.mail.ru//society/41444458/',
  'name': 'Что говорят предприниматели о карантине',
  'source': 'Коммерсантъ'},
 {'_id': '41444858',
  'date_time': '2020-04-18 16:09:09',
  'link': 'https://news.mail.ru//society/41444858/',
  'name': 'COVID-19 породил новую экологическую проблему (фото)',
  'source': 'Но

In [29]:
class LParser:
    '''парсер lenta.ru'''
    
    main_link = 'https://lenta.ru/'
    
    def __init__(self, agent):
        '''получение user agent'''

        self.header = agent
        
    def get_html(self):
        '''получаем доступ к странице'''

        response = requests.get(self.main_link, headers=self.header)
        root = html.fromstring(response.text)

        return root
    
    def get_news(self, root):
        '''название и ссылка'''
        
        main = "//section[contains(@class,'b-top7-for-main')]/div/div[contains(@class,'item')]"
        
        name_lists = root.xpath(f"{main}/h2/a/text()"), \
                    root.xpath(f"{main}/a/text()")
        
        names = []
        for name_list in name_lists:
            for name in name_list:
                names.append(name)
                
        links = root.xpath(f"{main}/a/@href")
        
        return names, links
    
    def edit_name(self, name):
        '''редактор имени'''

        pat = '\xa0'
        pattern = re.compile(pat)
        if pat in name:
            return re.sub(pattern, ' ', name)
        else:
            return name
        
    def time_source(self, link):
        '''собираем дату и источник'''

        response = requests.get(link, headers=self.header)
        root = html.fromstring(response.text)
        
        # источник
        try:
            source = root.xpath("//div[@class='b-label__credits']/text()")[0]
            Spattern1 = re.compile('.+/ ')
            Spattern2 = re.compile('.+')
            if '/' in source:
                source = re.sub(Spattern1, '', source)
            else:
                source = re.sub(Spattern2, 'Источник отсутствует', source)
        except IndexError as e:
            source = None
        
        # дата и время
        DTpattern = re.compile('\+[\d:]+')
        try:
            date_time = root.xpath("//time[@class='g-date']/@datetime")[0]
            date_time = re.sub(DTpattern, '', re.sub('T', ' ', date_time))
        except IndexError as e:
            date_time = None
        
        return source, date_time
    
    def get_id(self, link):
        '''собираем id'''

        pattern = re.compile('([/\d+]{10}/[\w]+)')
        return re.findall(pattern, link)
        
    def result(self):
        '''робираем данные'''
        
        root = self.get_html()
        news = self.get_news(root)
        result = []

        for n in range(len(news[0])):
            data_news = {}

            name = self.edit_name(news[0][n])
            if 'http' in news[1][n]:
                link = news[1][n]
            else:
                link = self.main_link + news[1][n]
            source = self.time_source(link)[0]
            date_time = self.time_source(link)[1]
            try:
                id = self.get_id(link)[0]
            except IndexError:
                id = None
            
            data_news['name'] = name
            data_news['link'] = link
            data_news['source'] = source
            data_news['date_time'] = date_time
            data_news['_id'] = id

            result.append(data_news)
        
        return result

In [30]:
Ldata = LParser(header).result()

pprint(Ldata)
print(f'\nКоличество объектов: {len(Ldata)}')

[{'_id': '2020/04/19/armata',
  'date_time': '2020-04-19 16:40:00',
  'link': 'https://lenta.ru//news/2020/04/19/armata/',
  'name': 'Россия испытала танк «Армата» в Сирии',
  'source': 'РИА Новости'},
 {'_id': '2020/04/19/prognoz',
  'date_time': '2020-04-19 17:29:43',
  'link': 'https://lenta.ru//news/2020/04/19/prognoz/',
  'name': 'Названы сроки возврата к нормальной жизни после эпидемии '
          'коронавируса в России',
  'source': 'РИА Новости'},
 {'_id': '2020/04/19/dog',
  'date_time': '2020-04-19 17:26:00',
  'link': 'https://lenta.ru//news/2020/04/19/dog/',
  'name': 'Мужчина нашел на шоссе собаку с трогательной запиской и расплакался',
  'source': 'Источник отсутствует'},
 {'_id': '2020/04/19/aurus',
  'date_time': '2020-04-19 17:11:06',
  'link': 'https://lenta.ru//news/2020/04/19/aurus/',
  'name': 'Стали известны сроки первой поставки машин из «Кортежа» Путина',
  'source': 'РИА Новости'},
 {'_id': '2020/04/19/chestno',
  'date_time': '2020-04-19 17:03:00',
  'link': '

In [31]:
mongo = ManagerMongoDB(Ldata).show_result()

pprint(mongo)
print(f'\nКоличество объектов: {len(mongo)}')

[{'_id': '41447768',
  'date_time': '2020-04-18 23:46:54',
  'link': 'https://news.mail.ru//society/41447768/',
  'name': 'Пасхальное богослужение в храме Христа Спасителя',
  'source': 'ТАСС'},
 {'_id': '41445652',
  'date_time': '2020-04-18 20:50:28',
  'link': 'https://news.mail.ru//society/41445652/',
  'name': 'Коронавирус COVID-19: главные события 18 апреля',
  'source': 'Новости Mail.ru'},
 {'_id': '41446112',
  'date_time': '2020-04-18 19:58:11',
  'link': 'https://news.mail.ru//economics/41446112/',
  'name': 'Туристический сезон в Турции может быть открыт в конце мая',
  'source': 'Коммерсантъ'},
 {'_id': '41444458',
  'date_time': '2020-04-18 17:36:29',
  'link': 'https://news.mail.ru//society/41444458/',
  'name': 'Что говорят предприниматели о карантине',
  'source': 'Коммерсантъ'},
 {'_id': '41444858',
  'date_time': '2020-04-18 16:09:09',
  'link': 'https://news.mail.ru//society/41444858/',
  'name': 'COVID-19 породил новую экологическую проблему (фото)',
  'source': 'Но

In [11]:
class YaParser:
    '''парсер news.mail.ru'''

    main_link = 'https://yandex.ru/news/'

    def __init__(self, agent):
        '''получение user agent'''

        self.header = agent
        
    def get_html(self):
        '''получаем доступ к странице'''

        response = requests.get(self.main_link, headers=self.header)
        root = html.fromstring(response.text)

        return root
    
    def get_news(self, root):
        '''сбор названия и ссылки'''

        main = "//div[@class='card__body']"
        
        # названия
        name_list = root.xpath(f"{main}/a/h2/span/text()")
        
        # ссылки
        path_lists = root.xpath(f"{main}/a/@href")
        
        # источник
        source = root.xpath(f"{main}/footer/div/a/span[@class='card__source-name']/text()")
        
        # время публикации
        date_time = root.xpath(f"{main}/footer/div/a/span[contains(@class,'sport-date')]/text()")
                                
        return name_list, path_lists, source, date_time
    
    def get_id(self, link):
        '''собираем id'''

        pattern = re.compile('id=(\d+)')
        
        return re.findall(pattern, link)[0]
        
    def result(self):
        '''собираем данные'''
        
        root = self.get_html()
        news = self.get_news(root)
        
        result = []
        
        for n in range(len(news[0])):
            data_news = {}

            name = news[0][n]
            link = news[1][n]
            source = news[2][n]
            date_time = f"{datetime.now().date()} {news[3][n]}"
            id = self.get_id(link)

            data_news['name'] = name
            data_news['link'] = link
            data_news['source'] = source
            data_news['date_time'] = date_time
            data_news['_id'] = id

            result.append(data_news)

        return result

In [12]:
while True:
    Yadata = YaParser(header).result()
    if len(Yadata) != 0:
        break

pprint(Yadata)
print(f'\nКоличество объектов: {len(Yadata)}')

[{'_id': '94919465',
  'date_time': '2020-04-19 15:00',
  'link': 'https://yandex.ru/news/story/Peskov_dopustil_vykhod_na_pik_po_koronavirusu_na_sleduyushhej_nedele--1ea274cfa03bb38d6637001f68ec0efa?lang=ru&stid=-quPKXcAHqigTZqva9Kv&t=1587323615&tt=true&persistent_id=94919465',
  'name': 'Песков допустил выход на пик по коронавирусу на следующей неделе',
  'source': 'РИА Новости'},
 {'_id': '94920274',
  'date_time': '2020-04-19 16:25',
  'link': 'https://yandex.ru/news/story/Siluanov_do_konca_goda_iz_FNB_potratyat_pochti_6_trln_rublej--1b51cb7e44aac8817f104971a359c6b4?lang=ru&stid=u3QnIkADsudK9vVPB2n3&t=1587323615&tt=true&persistent_id=94920274',
  'name': 'Силуанов: до конца года из ФНБ потратят почти 6 трлн рублей',
  'source': 'REGNUM'},
 {'_id': '94927827',
  'date_time': '2020-04-19 17:20',
  'link': 'https://yandex.ru/news/story/Tank_T-14_Armata_ispytali_v_Sirii--f0682c45362409204b999fd7ed2d238a?lang=ru&stid=cnB1ZEVIx7NdynH-E3DZ&t=1587323615&tt=true&persistent_id=94927827',
  'n

In [14]:
mongo = ManagerMongoDB(Yadata).show_result()

pprint(mongo)
print(f'\nКоличество объектов: {len(mongo)}')

[{'_id': '41447768',
  'date_time': '2020-04-18 23:46:54',
  'link': 'https://news.mail.ru//society/41447768/',
  'name': 'Пасхальное богослужение в храме Христа Спасителя',
  'source': 'ТАСС'},
 {'_id': '41445652',
  'date_time': '2020-04-18 20:50:28',
  'link': 'https://news.mail.ru//society/41445652/',
  'name': 'Коронавирус COVID-19: главные события 18 апреля',
  'source': 'Новости Mail.ru'},
 {'_id': '41446112',
  'date_time': '2020-04-18 19:58:11',
  'link': 'https://news.mail.ru//economics/41446112/',
  'name': 'Туристический сезон в Турции может быть открыт в конце мая',
  'source': 'Коммерсантъ'},
 {'_id': '41444458',
  'date_time': '2020-04-18 17:36:29',
  'link': 'https://news.mail.ru//society/41444458/',
  'name': 'Что говорят предприниматели о карантине',
  'source': 'Коммерсантъ'},
 {'_id': '41444858',
  'date_time': '2020-04-18 16:09:09',
  'link': 'https://news.mail.ru//society/41444858/',
  'name': 'COVID-19 породил новую экологическую проблему (фото)',
  'source': 'Но